# LangChain의 Single Agent (한글 Colab 실습)

이 노트북은 LangChain 튜토리얼([agents.ipynb](https://colab.research.google.com/github/langchain-ai/langchain/blob/master/docs/docs/tutorials/agents.ipynb)) 내용을 **한글**로 재구성한 실습 예제입니다.

## 목표
- ReAct 스타일의 **Single Agent**(LLM + Tool)를 LangGraph 프리빌트로 빠르게 만든다.
- **Wikipedia 검색 툴**(무키)과, 선택적으로 **Tavily 검색 툴**(API 키 필요)을 연결한다.
- `MemorySaver` 체크포인팅으로 **세션 기억**(thread_id)을 유지하고, `.stream()` 으로 단계별 출력을 관찰한다.


In [ ]:
# 0) 필수 라이브러리 설치
!pip -q install langchain langgraph langchain-community wikipedia
# (선택) Tavily 검색을 쓰려면 다음도 설치하세요
!pip -q install langchain-tavily
# (선택) OpenAI/Anthropic를 쓸 때
!pip -q install langchain-openai langchain-anthropic


## 1. 환경 변수 설정 (선택)
- **OpenAI** 를 쓴다면 `OPENAI_API_KEY` 필요
- **Anthropic** 를 쓴다면 `ANTHROPIC_API_KEY` 필요
- **Tavily** 검색을 쓴다면 `TAVILY_API_KEY` 필요

> 키가 없다면 본 노트북은 **Wikipedia 툴**만으로도 동작합니다.

In [ ]:
import os
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"
# os.environ["ANTHROPIC_API_KEY"] = "YOUR_ANTHROPIC_API_KEY"
# os.environ["TAVILY_API_KEY"] = "YOUR_TAVILY_API_KEY"

# (선택) LangSmith 트레이싱
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = "YOUR_LANGSMITH_KEY"
# os.environ["LANGCHAIN_PROJECT"] = "single-agent-ko-demo"


## 2. 모델 & 도구 준비
튜토리얼은 `create_react_agent` 로 **ReAct 프리빌트 에이전트**를 구성합니다.
- 모델은 OpenAI 또는 Anthropic 중 **사용 가능한 것 하나**를 선택하세요.
- 도구는 기본으로 **Wikipedia**를 사용하고, 키가 있다면 **Tavily**를 추가해봅니다.

In [ ]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools.wikipedia.tool import WikipediaQueryRun

use_openai = bool(os.environ.get("OPENAI_API_KEY"))
use_anthropic = bool(os.environ.get("ANTHROPIC_API_KEY"))

if use_openai:
    from langchain_openai import ChatOpenAI
    model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
elif use_anthropic:
    from langchain_anthropic import ChatAnthropic
    model = ChatAnthropic(model="claude-3-5-haiku-latest", temperature=0)
else:
    # 키가 전혀 없으면, 여기서는 에이전트 그래프만 구성하도록 안내 메시지를 던지고 종료합니다.
    # (Colab에서 키를 입력한 뒤 이 셀을 다시 실행하세요.)
    raise RuntimeError("OPENAI_API_KEY 또는 ANTHROPIC_API_KEY 중 하나를 환경변수로 설정하세요.")

# Wikipedia 툴 (무키)
wiki = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(lang="en"))
tools = [wiki]

# (선택) Tavily 검색 툴
try:
    if os.environ.get("TAVILY_API_KEY"):
        from langchain_tavily import TavilySearch
        tools.append(TavilySearch(max_results=2))
except Exception as e:
    print("Tavily 로드 생략:", e)

memory = MemorySaver()
agent = create_react_agent(model, tools, checkpointer=memory)
config = {"configurable": {"thread_id": "session-" + uuid.uuid4().hex[:8]}}
config

## 3. 스트리밍 실행
`.stream(..., stream_mode="values")` 형태로 각 스텝의 마지막 메시지를 확인합니다.
아래 예시는 **한국어 질문**으로 질의하고, 도구 호출(위키/탐색)이 필요하면 자동으로 수행합니다.

In [ ]:
prompt = "샌프란시스코의 대표적인 관광지는 어디인가요? 최신 정보가 있으면 검색해서 알려줘. 한국어로 답해줘."

for step in agent.stream({"messages": [{"role": "user", "content": prompt}]}, config, stream_mode="values"):
    msg = step["messages"][-1]
    # content가 list 또는 str일 수 있어 안전하게 처리
    content = msg.content if isinstance(msg.content, str) else str(msg.content)
    print(content)


## 4. 세션 기억 테스트 (thread_id)
같은 `config`(동일 thread_id)를 사용하면 앞선 대화 맥락이 이어집니다.

In [ ]:
follow_up = "방금 추천한 장소 중에서 가족 여행에 특히 좋은 곳 2곳만 골라 이유와 함께 다시 말해줘."
result = agent.invoke({"messages": [{"role": "user", "content": follow_up}]}, config)
print(result["messages"][-1].content)


## 5. 실습 과제
1) **자체 도구 추가**: 간단한 **수학 계산기** 함수를 파이썬로 만들고, LangChain 툴로 래핑하여 에이전트에 등록하세요. 예: `def add(a:int,b:int)->int`.

2) **제한/가드레일**: "위키피디아 결과가 없으면 Tavily를 쓰고, 둘 다 없으면 사과하고 중단" 규칙을 프롬프트 또는 커스텀 노드로 넣어보세요.

3) **LangSmith 트레이싱**: 트레이싱을 켜고, 툴 호출/프롬프트 체인을 시각화 해 보세요.

4) **한글 검색 품질 개선**: `WikipediaAPIWrapper(lang="ko")`로 교체하고, 같은 질문을 한국어/영어로 각각 실행하여 결과 차이를 비교하세요.


## 6. (선택) 간단한 커스텀 툴 추가 예시
아래는 기본적인 **덧셈 계산기**를 도구로 등록하는 스니펫입니다. (직접 실행하고 위 에이전트에 `tools.append(calc_tool)` 하세요.)

In [ ]:
from langchain.tools import tool

@tool
def add_two_numbers(a: int, b: int) -> int:
    """두 정수를 입력받아 합을 반환합니다."""
    return a + b

calc_tool = add_two_numbers
print(calc_tool.name, calc_tool.description)

## 참고
- LangChain Agents 튜토리얼(원문): `docs/docs/tutorials/agents.ipynb`
- LangGraph 프리빌트 ReAct 에이전트: `langgraph.prebuilt.create_react_agent`
- Wikipedia 도구: `langchain_community.tools.wikipedia`
- Tavily 검색: `langchain_tavily.TavilySearch`
- MemorySaver 체크포인팅: `langgraph.checkpoint.memory.MemorySaver`
